In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import plotly.express as px

# 1. Load dataset
df = pd.read_csv("Strawberry_Greenhouse.csv")

# 2. Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace('temparature', 'temperature').str.replace('irtemparature', 'irtemperature')

# 3. Parse timestamps
df['timestamp'] = pd.to_datetime(df['enqueuedtimeutc'], format='mixed', dayfirst=True)

# 4. Convert to numeric
for col in ['temperature', 'humidity', 'irtemperature']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 5. Drop rows with missing values
df.dropna(subset=['temperature', 'humidity', 'irtemperature'], inplace=True)

# 6. Feature Engineering
df['hour'] = df['timestamp'].dt.hour
df['day'] = df['timestamp'].dt.date
df['z_temp'] = zscore(df['temperature'])
df['z_hum'] = zscore(df['humidity'])
df['z_ir'] = zscore(df['irtemperature'])
df['outlier'] = (df[['z_temp', 'z_hum', 'z_ir']].abs() > 3).any(axis=1)

# 7. Summary Statistics per Device
device_summary = df.groupby('deviceid').agg(
    avg_temp=('temperature', 'mean'),
    avg_humidity=('humidity', 'mean'),
    avg_ir=('irtemperature', 'mean'),
    outlier_count=('outlier', 'sum'),
    total_records=('temperature', 'count')
)
device_summary['outlier_rate'] = device_summary['outlier_count'] / device_summary['total_records']
device_summary.reset_index(inplace=True)

# 8. Correlation Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(df[['temperature', 'humidity', 'irtemperature']].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.tight_layout()
plt.savefig("correlation_heatmap.png")
plt.close()

# 9. Time-Series Plot of All Sensors
plt.figure(figsize=(12, 6))
sns.lineplot(data=df[df['deviceid'] == df['deviceid'].unique()[0]], x='timestamp', y='temperature', label='Temp')
sns.lineplot(data=df[df['deviceid'] == df['deviceid'].unique()[0]], x='timestamp', y='humidity', label='Humidity')
plt.title("Sensor Readings Over Time (Sample Device)")
plt.xlabel("Time")
plt.ylabel("Value")
plt.legend()
plt.tight_layout()
plt.savefig("timeseries_sample_device.png")
plt.close()

# 10. Outlier Trends by Day
daily_outliers = df.groupby('day')['outlier'].sum()
plt.figure(figsize=(10, 5))
daily_outliers.plot(kind='bar', color='orange')
plt.title("Outlier Count per Day")
plt.xlabel("Date")
plt.ylabel("Outliers")
plt.tight_layout()
plt.savefig("outliers_per_day.png")
plt.close()

# 11. Interactive Correlation Plot (Plotly)
fig = px.scatter_matrix(df,
    dimensions=['temperature', 'humidity', 'irtemperature'],
    color='deviceid',
    title="Sensor Reading Scatter Matrix")
fig.write_html("scatter_matrix_interactive.html")

# 12. Save cleaned dataset
df.to_csv("cleaned_greenhouse_data.csv", index=False)

# 13. Save device summary
device_summary.to_csv("device_summary.csv", index=False)
